# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [2]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

✅ Librerías importadas.


In [3]:
# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [4]:
df_raw

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


In [5]:
conn = sqlite3.connect('../Datos/Educacion.bd')

In [6]:
df_raw.to_sql('Educacion',conn)

ValueError: Table 'Educacion' already exists.

In [7]:
cursor = conn.cursor()

In [8]:
pd.read_sql('SELECT * FROM Educacion', con=conn)

,index,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,None,None
1,1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,None,None
2,2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,None,None
3,3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,None,None
4,4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,...,0,0,0,None,0.83,None,9.93,4.47,20.01,53.45
14583,14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


In [9]:
def ejecutar_sql(query, conexion=conn):
  """
  Toma una consulta SQL, la ejecuta y devuelve el resultado
  en una tabla de pandas para una visualización clara.
  """
  df = pd.read_sql_query(query, conexion)
  # Usamos display() porque en Colab presenta las tablas de forma más elegante.
  display(df)

In [10]:
ejecutar_sql("SELECT departamento, COUNT (municipio) as total FROM Educacion GROUP BY departamento")

,departamento,total
0,Amazonas,143
1,Antioquia,1625
2,Arauca,91
3,"Archipiélago de San Andrés, Providencia y Sant...",24
4,Archipiélago de San Andrés. Providencia y Sant...,2
5,Atlántico,299
6,Bogotá D.C.,1
7,"Bogotá, D.C.",12
8,Bolívar,598
9,Boyacá,1599


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

In [11]:
ejecutar_sql("""
PRAGMA table_info(Educacion);
""")


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,a_o,TEXT,0,None,0
2,2,c_digo_municipio,TEXT,0,None,0
3,3,municipio,TEXT,0,None,0
4,4,c_digo_departamento,TEXT,0,None,0
5,5,departamento,TEXT,0,None,0
6,6,c_digo_etc,TEXT,0,None,0
7,7,etc,TEXT,0,None,0
8,8,poblaci_n_5_16,TEXT,0,None,0
9,9,tasa_matriculaci_n_5_16,TEXT,0,None,0


In [12]:
def ejecutar_sql(query, conexion=conn):
    """
    Ejecuta una consulta SQL. Si es un SELECT, muestra el resultado como tabla.
    Si no, ejecuta sin retorno (DROP, CREATE, etc.).
    """
    if query.strip().lower().startswith("select"):
        df = pd.read_sql_query(query, conexion)
        display(df)
    else:
        cursor = conexion.cursor()
        cursor.execute(query)
        conexion.commit()
        print("✅ Consulta ejecutada correctamente.")


In [13]:
ejecutar_sql("DROP TABLE IF EXISTS Educacion_Limpia")

✅ Consulta ejecutada correctamente.


In [14]:
ejecutar_sql("""
CREATE TABLE Educacion_Limpia AS
SELECT
    CAST(a_o AS INTEGER) AS anio,
    departamento,
    municipio,
    CAST(poblaci_n_5_16 AS REAL) AS poblacion,
    CAST(tasa_matriculaci_n_5_16 AS REAL) AS tasa_matriculacion,
    CAST(cobertura_neta AS REAL) AS cobertura_neta,
    CAST(aprobaci_n AS REAL) AS aprobacion,
    CAST(reprobaci_n AS REAL) AS reprobacion
FROM Educacion
WHERE a_o IS NOT NULL
  AND departamento IS NOT NULL
  AND municipio IS NOT NULL
  AND poblaci_n_5_16 IS NOT NULL
  AND tasa_matriculaci_n_5_16 IS NOT NULL
  AND cobertura_neta IS NOT NULL;
""")


✅ Consulta ejecutada correctamente.


In [15]:
pd.read_sql('SELECT * FROM Educacion_Limpia', con=conn)

,anio,departamento,municipio,poblacion,tasa_matriculacion,cobertura_neta,aprobacion,reprobacion
0,2023,Antioquia,Abriaquí,503.0,62.62,62.62,92.26,6.55
1,2023,Guaviare,El Retorno,4438.0,53.27,53.27,87.67,6.78
2,2023,Guaviare,Miraflores,2014.0,32.52,32.52,82.68,9.47
3,2023,Vaupés,Mitú,10986.0,59.57,59.57,90.71,5.34
4,2023,Vaupés,Caruru,1228.0,51.30,51.30,82.40,9.24
...,...,...,...,...,...,...,...,...
14446,2011,Antioquia,Angelópolis,1707.0,78.85,78.90,86.27,5.59
14447,2011,Antioquia,Andes,10244.0,84.45,84.50,95.46,0.58
14448,2011,Antioquia,Amalfi,5552.0,97.71,97.70,91.25,0.00
14449,2011,Antioquia,Amagá,6631.0,78.65,78.70,86.86,8.99


In [16]:
ejecutar_sql("DROP TABLE IF EXISTS Dim_Tiempo;")

✅ Consulta ejecutada correctamente.


In [17]:
ejecutar_sql("""
CREATE TABLE Dim_Tiempo AS
SELECT 
    anio,
    ROW_NUMBER() OVER (ORDER BY anio) AS id_tiempo
FROM (
    SELECT DISTINCT anio
    FROM Educacion_Limpia
    WHERE anio IS NOT NULL
)
""")


✅ Consulta ejecutada correctamente.


In [18]:
pd.read_sql('SELECT * FROM Dim_Tiempo;', con=conn)

,anio,id_tiempo
0,2011,1
1,2012,2
2,2013,3
3,2014,4
4,2015,5
5,2016,6
6,2017,7
7,2018,8
8,2019,9
9,2020,10


In [19]:
ejecutar_sql("DROP TABLE IF EXISTS Dim_Geografia;")

✅ Consulta ejecutada correctamente.


In [20]:
ejecutar_sql("""
CREATE TABLE Dim_Geografia AS
SELECT 
    departamento,
    municipio,
    ROW_NUMBER() OVER (ORDER BY departamento, municipio) AS id_geografia
FROM (
    SELECT DISTINCT departamento, municipio
    FROM Educacion_Limpia
    WHERE departamento IS NOT NULL AND municipio IS NOT NULL
);
""")


✅ Consulta ejecutada correctamente.


In [21]:
pd.read_sql('SELECT * FROM Dim_Geografia;', con=conn)

,departamento,municipio,id_geografia
0,Amazonas,El Encanto,1
1,Amazonas,La Chorrera,2
2,Amazonas,La Pedrera,3
3,Amazonas,La Victoria,4
4,Amazonas,Leticia,5
...,...,...,...
1121,Vaupés,Yavaraté,1122
1122,Vichada,Cumaribo,1123
1123,Vichada,La Primavera,1124
1124,Vichada,Puerto Carreño,1125


In [22]:
ejecutar_sql("DROP TABLE IF EXISTS Fact_Educacion;")

✅ Consulta ejecutada correctamente.


In [23]:
ejecutar_sql("""
CREATE TABLE Fact_Educacion AS
SELECT 
    t.id_tiempo,
    g.id_geografia,
    AVG(e.poblacion) AS poblacion,
    AVG(e.tasa_matriculacion) AS tasa_matriculacion,
    AVG(e.cobertura_neta) AS cobertura_neta,
    AVG(e.aprobacion) AS aprobacion,
    AVG(e.reprobacion) AS reprobacion
FROM Educacion_Limpia e
JOIN Dim_Tiempo t ON e.anio = t.anio
JOIN Dim_Geografia g ON e.departamento = g.departamento AND e.municipio = g.municipio
GROUP BY t.id_tiempo, g.id_geografia;
""")


✅ Consulta ejecutada correctamente.


In [24]:
pd.read_sql('SELECT * FROM Fact_Educacion;', con=conn)

,id_tiempo,id_geografia,poblacion,tasa_matriculacion,cobertura_neta,aprobacion,reprobacion
0,1,1,1543.0,31.43,31.40,95.43,2.29
1,1,2,1135.0,54.45,54.50,96.53,0.42
2,1,3,1433.0,72.99,73.00,95.96,0.09
3,1,5,11410.0,108.20,108.20,90.02,4.04
4,1,8,488.0,40.78,40.80,94.55,0.00
...,...,...,...,...,...,...,...
14446,13,1122,561.0,47.95,47.95,95.55,0.00
14447,13,1123,25357.0,49.85,49.85,76.05,16.45
14448,13,1124,2659.0,95.94,95.94,83.33,10.23
14449,13,1125,4641.0,144.52,144.52,80.93,13.26


### Porcentaje de escolaridad (tasa de matrícula 5–16 años) por municipio

In [25]:
ejecutar_sql("""
SELECT 
    g.departamento,
    g.municipio,
    ROUND(f.tasa_matriculacion, 2) AS tasa_matriculacion,
    ROUND(f.cobertura_neta, 2) AS cobertura_neta
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
ORDER BY f.tasa_matriculacion DESC
LIMIT 10;
""")


,departamento,municipio,tasa_matriculacion,cobertura_neta
0,Atlántico,Puerto Colombia,279.03,139.67
1,Atlántico,Puerto Colombia,273.80,264.54
2,Atlántico,Puerto Colombia,266.50,257.24
3,Cundinamarca,Cota,240.70,238.62
4,Cundinamarca,Cota,239.34,199.98
5,Cundinamarca,Cota,236.70,233.99
6,Cundinamarca,Cota,197.50,196.15
7,Cundinamarca,Cota,195.16,193.44
8,Santander,Sabana de Torres,194.87,94.17
9,Cundinamarca,Cota,193.34,191.50


### Comparación de rendimiento educativo por municipio (aprobación vs. reprobación)

In [26]:
ejecutar_sql("""
SELECT 
    g.departamento,
    g.municipio,
    ROUND(f.tasa_matriculacion, 2) AS tasa_matriculacion,
    ROUND(f.cobertura_neta, 2) AS cobertura_neta
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
ORDER BY f.tasa_matriculacion DESC
LIMIT 10;
""")


,departamento,municipio,tasa_matriculacion,cobertura_neta
0,Atlántico,Puerto Colombia,279.03,139.67
1,Atlántico,Puerto Colombia,273.80,264.54
2,Atlántico,Puerto Colombia,266.50,257.24
3,Cundinamarca,Cota,240.70,238.62
4,Cundinamarca,Cota,239.34,199.98
5,Cundinamarca,Cota,236.70,233.99
6,Cundinamarca,Cota,197.50,196.15
7,Cundinamarca,Cota,195.16,193.44
8,Santander,Sabana de Torres,194.87,94.17
9,Cundinamarca,Cota,193.34,191.50


###  Departamentos con mejor cobertura neta

In [27]:
ejecutar_sql("""
SELECT 
    g.departamento,
    ROUND(AVG(f.cobertura_neta), 2) AS cobertura_promedio
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
GROUP BY g.departamento
ORDER BY cobertura_promedio DESC
LIMIT 10;
""")


,departamento,cobertura_promedio
0,Bogotá D.C.,95.89
1,Quindio,94.58
2,Sucre,93.85
3,Cesar,93.81
4,Magdalena,93.28
5,Meta,90.74
6,Tolima,89.24
7,"Bogotá, D.C.",88.96
8,Casanare,88.58
9,Cundinamarca,88.55


## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

In [28]:
ejecutar_sql("""
SELECT 
    g.departamento,
    g.municipio,
    ROUND(f.tasa_matriculacion, 2) AS porcentaje_escolaridad
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
JOIN Dim_Tiempo t ON f.id_tiempo = t.id_tiempo
WHERE t.anio = (
    SELECT MAX(anio) FROM Dim_Tiempo
)
ORDER BY porcentaje_escolaridad DESC
LIMIT 10;
""")

,departamento,municipio,porcentaje_escolaridad
0,Cundinamarca,Cota,186.72
1,Cundinamarca,La Calera,160.56
2,Cundinamarca,Fúquene,155.74
3,Atlántico,Puerto Colombia,155.14
4,Quindio,Salento,151.46
5,Vichada,Puerto Carreño,144.52
6,Cundinamarca,Tenjo,143.05
7,Nariño,Cumbitara,127.64
8,Antioquia,La Estrella,125.77
9,Cundinamarca,Subachoque,125.75


2. ¿Cómo compararía el rendimiento educativo por municipios?
   Municipios con alta aprobación y baja reprobación muestran un mejor desempeño educativo. 

In [29]:
ejecutar_sql("""
SELECT 
    g.departamento,
    g.municipio,
    ROUND(f.aprobacion, 2) AS aprobacion,
    ROUND(f.reprobacion, 2) AS reprobacion,
    t.anio
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
JOIN Dim_Tiempo t ON f.id_tiempo = t.id_tiempo
WHERE t.anio = (SELECT MAX(anio) FROM Dim_Tiempo)
ORDER BY aprobacion DESC
LIMIT 10;
""")


,departamento,municipio,aprobacion,reprobacion,anio
0,Amazonas,La Victoria,100.00,0.00,2023
1,Vaupés,Taraira,100.00,0.00,2023
2,Nariño,Nariño,99.78,0.00,2023
3,Amazonas,Puerto Arica,99.62,0.38,2023
4,Cundinamarca,Tibirita,99.50,0.00,2023
5,Nariño,Providencia,99.42,0.15,2023
6,Nariño,Pupiales,99.29,0.04,2023
7,Cundinamarca,La Palma,98.81,0.00,2023
8,Cundinamarca,Gama,98.74,0.00,2023
9,Nariño,La Tola,98.30,0.00,2023


In [30]:
ejecutar_sql("""
SELECT 
    g.departamento,
    g.municipio,
    ROUND(f.aprobacion, 2) AS aprobacion,
    ROUND(f.reprobacion, 2) AS reprobacion,
    t.anio
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
JOIN Dim_Tiempo t ON f.id_tiempo = t.id_tiempo
WHERE t.anio = (SELECT MAX(anio) FROM Dim_Tiempo)
ORDER BY aprobacion ASC
LIMIT 10;
""")

,departamento,municipio,aprobacion,reprobacion,anio
0,Vaupés,Papunaua,0.00,0.00,2023
1,Guainía,Puerto Colombia,70.24,22.24,2023
2,Guainía,San Felipe,72.89,11.11,2023
3,Vichada,Cumaribo,76.05,16.45,2023
4,Huila,La Argentina,77.46,16.36,2023
5,Valle del Cauca,Ansermanuevo,77.64,11.70,2023
6,Guainía,Cacahual,77.68,14.68,2023
7,Amazonas,Puerto Nariño,78.27,12.06,2023
8,Guainía,La Guadalupe,78.95,15.79,2023
9,Antioquia,Hispania,79.14,7.65,2023


3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

In [31]:
ejecutar_sql("""
SELECT 
    g.departamento,
    ROUND(AVG(f.cobertura_neta), 2) AS cobertura_promedio,
    t.anio
FROM Fact_Educacion f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
JOIN Dim_Tiempo t ON f.id_tiempo = t.id_tiempo
WHERE t.anio = (SELECT MAX(anio) FROM Dim_Tiempo)
GROUP BY g.departamento, t.anio
ORDER BY cobertura_promedio DESC
LIMIT 10;
""")


,departamento,cobertura_promedio,anio
0,Quindio,95.08,2023
1,Vichada,93.55,2023
2,"Bogotá, D.C.",92.40,2023
3,Tolima,90.29,2023
4,Sucre,89.71,2023
5,Bolívar,89.64,2023
6,Meta,89.29,2023
7,Magdalena,88.81,2023
8,"Archipiélago de San Andrés, Providencia y Sant...",88.22,2023
9,Cundinamarca,87.98,2023


MODELO ESTRELLA RESULTANTE
Tabla de Hechos: Fact_Educacion

id_tiempo, id_geografia, poblacion, tasa_matriculacion, cobertura_neta, aprobacion, reprobacion

Dim_Tiempo: id_tiempo, anio

Dim_Geografia: id_geografia, departamento, municipio

In [32]:
# Cerramos la conexión
conn.close()
print("Conexión a la base de datos cerrada.")

Conexión a la base de datos cerrada.
